In [34]:
# criar um navegador
# importar base de dados

# para cada produto eu vou:
    #procurar esse produto no google shopping
        #verificar se está dentro da faixa de preço
    #procurar esse produto no buscape
        #verificar se está dentro da faixa de preço

#salvcar as ofertas em um dataframe
#exportar pro excel o resultado
#enviar pro e-mail

In [35]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from pprint import pprint
import re

nav = webdriver.Chrome()
nav.get("https://www.google.com")

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 13 128gb,mini watch,3000,4000
1,samsung z flip,usado casa,2500,4000
2,rtx 3060,pichau,2000,4000


### Funções verificação de termos banidos e de palavras obrigatórias

In [36]:
def verificar_termos_banidos(lista_termo_banido, nome):
# analisar se não tem nenhum termo banido 
    tem_termos_banidos = False
    for palavra in lista_termo_banido:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos


def verificar_palavras_obrigatórias(lista_termo_nome, nome):
# analisar se o nome do produto corresponde com o termo pedido
    tem_todos_termos_pedidos = True
    for palavra in lista_termo_nome:
        if palavra not in nome:
            tem_todos_termos_pedidos = False
    return tem_todos_termos_pedidos

### Busca no Google Shoping

In [37]:
def busca_google_shopping(nav, produto, termo_banido, preco_min, preco_max):

    import time

    preco_min = float(preco_min)
    preco_max = float(preco_max)

    produto = produto.lower()
    lista_termo_nome = produto.split(" ")
    termo_banido = termo_banido.lower()
    lista_termo_banido = termo_banido.split(" ")

    lista_ofertas_google_shopping = []
    #entrar no google
    nav.get('https://www.google.com')
    time.sleep(1)

    #procurar esse produto no google shopping
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/div/textarea').send_keys(produto, Keys.ENTER)
    time.sleep(1)

        #entrar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem') + nav.find_elements(By.CLASS_NAME, 'GKS7s')

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break
        time.sleep(1)

        #pegar as informações do produto

    lista_resultado = nav.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in lista_resultado:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()

        # analisar se não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termo_banido, nome)

        # analisar se o nome do produto corresponde com o termo pedido
        tem_todos_termos_pedidos = verificar_palavras_obrigatórias(lista_termo_nome, nome)

        # selecionar só os elementos que entram nos termos pedidos (não ter banido e estar no nome)
        import re # biblioteca importada para resolver o erro do coméntário abaixo
        
        if tem_todos_termos_pedidos == True and tem_termos_banidos == False:
        
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")

            # Adicione uma verificação para garantir que 'preco' não está vazio
            if preco:
                preco = float(re.sub(r'[^\d.]', '', preco))
            else:
                preco = 0.0  # Ou outro valor padrão que faça sentido no seu contexto

            # se o preço tá entre o preço mínimo e o máximo
            if preco_min <= preco <= preco_max:
            
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                # print(nome, preco, link)
                lista_ofertas_google_shopping.append((nome, preco, link))

    return lista_ofertas_google_shopping

### Busca no Buscapé

In [38]:

def busca_buscape(nav, produto, termo_banido, preco_min, preco_max):

    import time
    # tratar valores   

    lista_oferta_buscape = []

    produto = produto.lower()
    lista_termo_nome = produto.split(" ")
    termo_banido = termo_banido.lower()
    lista_termo_banido = termo_banido.split(" ")

    preco_min = float(preco_min)
    preco_max = float(preco_max)

    # entrar no buscapé e buscar os produtos
    nav.get('https://www.buscape.com.br/')
    nav.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(produto, Keys.ENTER)

        # Função para esperar a página carregar completamente
    # while len(nav.find_element(By.CLASS_NAME, "Select_Select__1HNob")) < 1:
    #     time.sleep(0.1)

    time.sleep(1)
    
    # pegar e tratar os resultados
    lista_resultado = nav.find_elements(By.CLASS_NAME ,'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultado:
        nome = resultado.find_element(By.CLASS_NAME, "ProductCard_ProductCard_Name__U_mUQ").text
        nome = nome.lower()

        # analisar se não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termo_banido, nome)

        # analisar se o nome do produto corresponde com o termo pedido
        tem_todos_termos_pedidos = verificar_palavras_obrigatórias(lista_termo_nome, nome)
        import re # biblioteca importada para resolver o erro do coméntário abaixo
        
        if tem_todos_termos_pedidos == True and tem_termos_banidos == False:
                
            preco = resultado.find_element(By.CLASS_NAME, "Text_MobileHeadingS__HEz7L").text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")

            ## Solução alternativa para caso de um erro irrelevante, apenas vai ignorar
            ## o erro e continuar o código
            # try:
            #     "Todo o código"
            # except:
            #     continue

            # Adicione uma verificação para garantir que 'preco' não está vazio
            if preco:
                preco = float(re.sub(r'[^\d.]', '', preco))
            else:
                preco = 0.0  # Ou outro valor padrão que faça sentido no seu contexto

            # se o preço tá entre o preço mínimo e o máximo
            if preco_min <= preco <= preco_max:
                link = resultado.get_attribute("href")
                lista_oferta_buscape.append((nome, preco, link))
                
    return lista_oferta_buscape
            
    # ver se tem ou não os termos banidos
    # verificar o preço minimo e o preço máximo

### Resultado

In [39]:
# tabela_ofertas = pd.DataFrame()

# for linha in tabela_produtos.index:

#     produto = tabela_produtos.loc[linha, "Nome"]
#     termo_banido = tabela_produtos.loc[linha, "Termos banidos"]
#     preco_min = tabela_produtos.loc[linha, "Preço mínimo"]
#     preco_max = tabela_produtos.loc[linha, "Preço máximo"]
#     lista_ofertas = []

#     lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termo_banido, preco_min, preco_max)

#     if lista_ofertas_google_shopping:
#         tabela_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=["Produto", "Preço", "Link"])
#         tabela = pd.DataFrame()
#         tabela.to_excel(f"tabela_{produto[:6]}")    
#     else:
#         lista_ofertas_google_shopping = None
#     display(tabela_shopping)


#     lista_oferta_buscape = busca_buscape(nav, produto, termo_banido, preco_min, preco_max)
#     if lista_oferta_buscape:
#         tabela_buscape = pd.DataFrame(lista_oferta_buscape, columns=["Produto", "Preço", "Link"])
#         tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
#         tabela = pd.DataFrame()
#         tabela.to_excel(f"tabela_{produto[:6]}")  

#     else:
#         lista_oferta_buscape = None
#     display(tabela_buscape)

# # display(tabela_ofertas)


### Fazendo o for percorrendo toda a lista e exportando pro excel

In [40]:
import os
excel_files = []

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termo_banido = tabela_produtos.loc[linha, "Termos banidos"]
    preco_min = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_max = tabela_produtos.loc[linha, "Preço máximo"]

    # Extrair o primeiro e o segundo nome (caso existam)
    primeiro_nome, segundo_nome = produto.split(maxsplit=1) if produto else ("", "")

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termo_banido, preco_min, preco_max)

    if lista_ofertas_google_shopping:
        tabela_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=["Produto", "Preço", "Link"])
        excel_filename = f"tabela_{primeiro_nome}_{segundo_nome}.xlsx"

        # Salvar o DataFrame como Excel
        tabela_shopping.to_excel(excel_filename, index=False)

        # Adicionar o caminho do arquivo Excel à lista
        excel_files.append(os.path.abspath(excel_filename))
        display(tabela_shopping)
    else:
        tabela_shopping = None
        print(f"Nenhuma oferta encontrada para {produto} no Google Shopping.")

    lista_oferta_buscape = busca_buscape(nav, produto, termo_banido, preco_min, preco_max)

    if lista_oferta_buscape:
        tabela_buscape = pd.DataFrame(lista_oferta_buscape, columns=["Produto", "Preço", "Link"])
        excel_filename = f"tabela__buscapé_{primeiro_nome}_{segundo_nome}.xlsx"

        # Salvar o DataFrame como Excel
        tabela_buscape.to_excel(excel_filename, index=False)

        # Adicionar o caminho do arquivo Excel à lista
        excel_files.append(os.path.abspath(excel_filename))
        display(tabela_buscape)
    else:
        tabela_shopping = None
        print(f"Nenhuma oferta encontrada para {produto} no Buscapé.")


,Produto,Preço,Link
0,"iphone 13 apple meia-noite, 128gb",3799.00,https://www.google.com/url?url=https://www.ama...
1,usado: iphone 13 128gb meia noite muito bom - ...,3077.94,https://www.google.com/url?url=https://www.mag...
2,vitrine iphone 13 azul 128gb,3863.99,https://www.google.com/url?url=https://www.car...
3,usado: iphone 13 128gb azul excelente - trocaf...,3275.74,https://www.google.com/url?url=https://www.mag...
4,usado: iphone 13 128gb estelar excelente - tro...,3275.74,https://www.google.com/url?url=https://www.mag...
5,iphone 13 128gb,3699.00,https://www.google.com/url?url=https://www.goi...
6,apple iphone 13 128gb 5g - vitrine - tela supe...,3483.80,https://www.google.com/url?url=https://www.hor...
7,"usado: iphone 13 128gb, estelar - excelente",3809.00,https://www.google.com/url?url=https://www.kab...
8,iphone 13 128gb meia noite excelente,3428.10,https://www.google.com/url?url=https://www.tro...
9,apple iphone 13 (128gb) preto,3002.65,https://www.google.com/url?url=https://mobilev...


,Produto,Preço,Link
0,smartphone apple iphone 13 128gb câmera dupla,3799.0,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 13 usado 128gb câmera ...,3221.1,https://www.buscape.com.br/celular/smartphone-...
2,smartphone apple iphone 13 pro usado 128gb câm...,3959.1,https://www.buscape.com.br/celular/smartphone-...
3,smartphone apple iphone 13 vermelho 128gb câme...,3999.0,https://www.buscape.com.br/celular/pre-venda-s...


,Produto,Preço,Link
0,smartphone samsung galaxy z flip4 5g 128gb azu...,3392.01,https://www.google.com/url?url=https://www.mag...
1,"smartphone samsung galaxy z flip5 5g, 256gb, 8...",3599.10,https://www.google.com/url?url=https://www.kab...
2,smartphone samsung galaxy z flip 256gb 8gb ram...,2868.50,https://www.google.com/url?url=https://zhiyund...
3,smartphone samsung galaxy z flip5 5g tela dobr...,3699.00,https://www.google.com/url?url=https://www.bus...
4,samsung galaxy z flip 3 128gb 8gb ram tela de ...,3844.50,https://www.google.com/url?url=https://hayloub...
5,novo samsung galaxy z flip4 sm-f721w - 128gb -...,2859.93,https://www.google.com/url?url=https://www.eba...
6,celular samsung galaxy z flip 4 128gb novo em ...,3944.75,https://www.google.com/url?url=https://www.eba...
7,samsung galaxy z flip 4 8gb+512gb purple unlocked,2655.00,https://www.google.com/url?url=https://tiendam...
8,"telefone samsung galaxy z flip f700f roxo 6,7""...",3898.45,https://www.google.com/url?url=https://www.eba...
9,samsung galaxy z flip 4 8gb+512gb gray unlocked,2655.00,https://www.google.com/url?url=https://tiendam...


,Produto,Preço,Link
0,smartphone samsung galaxy z flip4 5g sm-f721u ...,3099.0,https://www.buscape.com.br/celular/smartphone-...
1,smartphone samsung galaxy z flip4 5g sm-f721u ...,3768.9,https://www.buscape.com.br/celular/smartphone-...


,Produto,Preço,Link
0,placa de vídeo galax geforce rtx 3060 1-click ...,2368.00,https://www.google.com/url?url=https://www.ama...
1,placa de vídeo asus geforce rtx 3060 dual oc 1...,3200.00,https://www.google.com/url?url=https://www.mer...
2,"placa de vídeo geforce rtx 3060 1-click oc, 12...",2229.90,https://www.google.com/url?url=https://www.mag...
3,placa de vídeo ventus 2x oc rtx 3060 8gb gddr6...,2333.58,https://www.google.com/url?url=https://shopee....
4,placa de vídeo gainward geforce rtx 3060 pegas...,2176.00,https://www.google.com/url?url=https://www.car...
5,placa de video colorful igame geforce rtx 3060...,2463.12,https://www.google.com/url?url=https://www.kab...
6,placa de video pcyes nvidia geforce rtx 3060 t...,2646.94,https://www.google.com/url?url=https://www.all...
7,placa de vídeo gainward ghost geforce rtx 3060...,2118.94,https://www.google.com/url?url=https://www.ext...
8,placa de vídeo geforce rtx 3060 v2 dual 12gb g...,2560.50,https://www.google.com/url?url=https://netcomp...
9,placa de vídeo gigabyte geforce rtx 3060 visio...,3092.96,https://www.google.com/url?url=https://br.octo...


,Produto,Preço,Link
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,3666.97,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 3060 12 gb g...,2272.61,https://www.buscape.com.br/placa-de-video/plac...
2,placa de video nvidia geforce rtx 3060 12 gb g...,2049.00,https://www.buscape.com.br/placa-de-video/plac...
3,placa vídeo msi geforce rtx 3060 12gb ventus 2...,2482.52,https://www.buscape.com.br/placa-de-video/plac...
4,placa de video nvidia geforce rtx 3060 12 gb g...,3699.90,https://www.buscape.com.br/placa-de-video/plac...
5,placa video geforce rtx 3060 ti twin x2 8gb gd...,3245.90,https://www.buscape.com.br/lead?oid=1168557678...
6,placa video geforce rtx 3060 ti twin x2 8gb gd...,3245.90,https://www.buscape.com.br/lead?oid=1168673817...
7,placa de vídeo galax geforce rtx 3060 12gb lhr...,2036.03,https://www.buscape.com.br/lead?oid=530254390&...
8,gigabyte placa de vídeo geforce rtx 3060 gamin...,2888.88,https://www.buscape.com.br/lead?oid=546338771&...
9,placa de vídeo gigabyte rtx 3060 vision oc 12g...,2551.16,https://www.buscape.com.br/lead?oid=867967780&...


### Mandando pro E-mail

In [41]:
import win32com.client as win32

if len(tabela_shopping) > 0:
    outlook = win32.Dispatch('outlook.application')

    mail = outlook.CreateItem(0)
    mail.To = "leumer.cruz@yahoo.com.br"
    mail.Subject = "Produto encontrado na faixa de Preço desejado"
    mail.Body = '''Prezado(a) Leumer,

Espero que esteja bem.

Anexo a este e-mail, segue a relação dos produtos solicitados que se encontram dentro da faixa de preço desejada.

Em caso de dúvidas ou necessidade de mais informações, estou à disposição.

Atenciosamente,

Neylor'''


     # Anexar todos os arquivos Excel à mensagem de e-mail
    for excel_file in excel_files:
        mail.Attachments.Add(excel_file)

    # Enviar e-mail
    mail.Send()
    print("E-mail enviado com os arquivos Excel anexados")

else:
    print("Nenhum arquivo Excel para enviar.")

E-mail enviado com os arquivos Excel anexados
